In [2]:
import pandas as pd
df=pd.read_csv('../database/inventario_reportes.csv', delimiter=",")

In [3]:
import re
def clean_text(text):
    text=str(text).lower()
    text=re.sub(r'[^\w\s]', '', text)
    return text.strip()

df['Nombre_limpio']=df['REPORTE'].apply(clean_text)
df['Descripcion_limpia']=df['DESCRIPCION_ESP'].apply(clean_text)

In [4]:
from sentence_transformers import SentenceTransformer

model=SentenceTransformer('../models/paraphrase-multilingual-MiniLM-L12-v2/')


df['texto_completo']=df['Nombre_limpio'] + ". " + df['Descripcion_limpia']
embeddings=model.encode(df['texto_completo'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
import faiss
import numpy as np


dimension=embeddings.shape[1]
index=faiss.IndexFlatL2(dimension)
index.add(embeddings.astype(np.float32))

In [6]:
def buscar_registros(consulta, top_k=5):
    consulta_limpia=clean_text(consulta)
    embedding_consulta=model.encode([consulta_limpia])
    
    distancias, indices=index.search(embedding_consulta.astype(np.float32), top_k)

    resultados=df.iloc[indices[0]].copy()
    resultados['similitud']=1 - distancias[0] / 4
    return resultados.sort_values(by='similitud', ascending=False)

In [7]:
resultados=buscar_registros("IPAB")

# Luego, usa los nombres correctos. Ejemplo:
print(resultados[['ID_INVENTARIO_REPORTE', 'CLAVE_ENTIDADREGULADA', 'REPORTE', 'DESCRIPCION_ESP', 'similitud']])

      ID_INVENTARIO_REPORTE CLAVE_ENTIDADREGULADA          REPORTE  \
736                    1303                    BM              COM   
747                     124                    BM  IPAB DEFINITIVO   
744                    1305                    BM               CR   
734                    1304                    BM              CNR   
1449                    418                   SOF             ICAP   

                 DESCRIPCION_ESP  similitud  
736         Tabla IPAB Comercial  -1.429139  
747   Cuotas IPAB Saldos Diarios  -1.680683  
744                Tabla IPAB CR  -1.798694  
734               Tabla IPAB CNR  -1.966000  
1449  Reporte Interno SOFOM ICAP  -2.184722  
